In [3]:
import numpy
import sys
!{sys.executable} -m pip install chaospy==3.2.0
import chaospy
from scipy.integrate import odeint

     |████████████████████████████████| 236 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 30.5 MB/s eta 0:00:01


In [4]:
class CoffeeCup:
    def __init__(self, time_points, max_time):
        self.time_points = time_points
        self.max_time = max_time

    def solve(self, T_0,kappa,T_env):
        time = numpy.linspace(0, self.time_points, self.max_time)  # Minutes

        # The equation describing the model
        def f(T, time, kappa, T_env):
            return -kappa * (T - T_env)

        # Solving the equation by integration
        temperature = odeint(f, T_0, time, args=(kappa, T_env))[:, 0]
        return temperature
    def exact(self, U_0, a):
        return U_0 * numpy.exp(a * self.x)

In [5]:
def run_solver(kappa, T_env):

    max_time, time_points, T_0 = 150, 200, 95
    model = CoffeeCup(time_points, max_time)
    solutions = model.solve(T_0, kappa, T_env)
    return [solutions[10],solutions[-1]]


In [27]:
order = 1

In [28]:
kappa_dist ,T_env_dist = chaospy.Uniform(0.025, 0.075), chaospy.Uniform(15, 25)
joint = chaospy.J(kappa_dist, T_env_dist)
ort = chaospy.orth_ttr(order,joint)
print(ort)

#kappa_dist     = chaospy.Normal(0, 1)
#T_env_dist     = chaospy.Normal(0, 1)


[1.0 q1-20.0 q0-0.05]


In [29]:
kappa_dist ,T_env_dist = chaospy.Uniform(0.025, 0.075), chaospy.Normal(15, 25)
joint = chaospy.J(kappa_dist, T_env_dist)
ort = chaospy.orth_ttr(order,joint)
print(ort)

[1.0 q1-15.0 q0-0.05]


In [32]:
joint = chaospy.J(kappa_dist, T_env_dist)
nodes, weights = chaospy.generate_quadrature(order, joint, rule="gaussian")
gauss_evals = numpy.array([run_solver(node[0],node[1]) for node in nodes.T])
quad = chaospy.fit_quadrature(ort, nodes, weights, gauss_evals)

In [33]:
print(quad)

[0.47925210777509747*q1-552.2992450088595*q0+77.08601218526006
 0.9995916106868575*q1-2.2494886167924415*q0+0.1512713403519467]
